<a href="https://colab.research.google.com/github/Wolf-of-github/AES-demonstration/blob/main/people_track_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!rm -rf /content/gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install -U insightface onnxruntime-gpu

In [ ]:
!pip install -U albumentations

In [4]:
!pip install tqdm

In [ ]:
# Uninstall current version
!pip uninstall -y torch torchvision torchaudio

# Install compatible version
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

# Reinstall insightface
!pip install -U insightface

In [ ]:
!pip install -U insightface

In [7]:
!pip install onnxruntime-gpu

In [5]:
import cv2
import os
import numpy as np
from insightface.app import FaceAnalysis
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# === Configuration ===
video_path = "/content/gdrive/MyDrive/Project Anthro/Face Recognition/input.mp4"
output_dir = "/content/gdrive/MyDrive/Project Anthro/Face Recognition/faces"
frame_interval = 10
similarity_threshold = 0.5  # Lower = stricter match
person_id_counter = 0
known_embeddings = []  # [(embedding, folder_name)]

# Setup face detector (GPU)
app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
app.prepare(ctx_id=0)

# Create output folder
os.makedirs(output_dir, exist_ok=True)

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("❌ ERROR: Could not open video.")

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"✅ Total frames in video: {total_frames}")

frame_idx = 0
saved_count = 0

# Progress bar
with tqdm(total=total_frames, desc="Processing video") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            height, width = frame.shape[:2]
            faces = app.get(frame)

            for i, face in enumerate(faces):
                x1, y1, x2, y2 = map(int, face.bbox)

                if x1 < 0 or y1 < 0 or x2 > width or y2 > height:
                    continue

                # Get embedding
                emb = face.embedding.reshape(1, -1)
                match_found = False

                for known_emb, folder_name in known_embeddings:
                    sim = cosine_similarity(emb, known_emb.reshape(1, -1))[0][0]
                    if sim > similarity_threshold:
                        person_folder = os.path.join(output_dir, folder_name)
                        match_found = True
                        break

                if not match_found:
                    person_id_counter += 1
                    folder_name = f"person_{person_id_counter}"
                    person_folder = os.path.join(output_dir, folder_name)
                    os.makedirs(person_folder, exist_ok=True)
                    known_embeddings.append((emb, folder_name))

                crop = frame[y1:y2, x1:x2]
                face_filename = f"face_{frame_idx}_{i}.jpg"
                cv2.imwrite(os.path.join(person_folder, face_filename), crop)
                saved_count += 1

        frame_idx += 1
        pbar.update(1)

cap.release()
print(f"✅ Finished. Saved {saved_count} face crops into {person_id_counter} unique person folders.")

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

Processing video: 100%|██████████| 2000/2000 [01:30<00:00, 22.08it/s]

✅ Finished. Saved 573 face crops into 83 unique person folders.


Trying to avoid back facing people and side profiled people

In [6]:
import cv2
import os
import numpy as np
from insightface.app import FaceAnalysis
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# === Configuration ===
video_path = "/content/gdrive/MyDrive/Project Anthro/Face Recognition/input.mp4"
output_dir = "/content/gdrive/MyDrive/Project Anthro/Face Recognition/faces"
frame_interval = 10
similarity_threshold = 0.5  # Lower = stricter match
person_id_counter = 0
known_embeddings = []  # [(embedding, folder_name)]

# Setup face detector (GPU)
app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
app.prepare(ctx_id=0)

# Create output folder
os.makedirs(output_dir, exist_ok=True)

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("❌ ERROR: Could not open video.")

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"✅ Total frames in video: {total_frames}")

frame_idx = 0
saved_count = 0

# Progress bar
with tqdm(total=total_frames, desc="Processing video") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            height, width = frame.shape[:2]
            faces = app.get(frame)

            for i, face in enumerate(faces):
                x1, y1, x2, y2 = map(int, face.bbox)
                if x1 < 0 or y1 < 0 or x2 > width or y2 > height:
                    continue

                # === Orientation check: skip back or profile faces ===
                kps = face.kps
                if kps is None or len(kps) < 5:
                    continue  # Landmark extraction failed → likely turned away

                left_eye, right_eye = kps[0], kps[1]
                eye_dx = abs(left_eye[0] - right_eye[0])
                eye_dy = abs(left_eye[1] - right_eye[1])
                eye_distance = np.hypot(eye_dx, eye_dy)

                if eye_distance < 10:  # Too close → likely not frontal
                    continue
                # === End orientation check ===

                # Get embedding
                emb = face.embedding.reshape(1, -1)
                match_found = False

                for known_emb, folder_name in known_embeddings:
                    sim = cosine_similarity(emb, known_emb.reshape(1, -1))[0][0]
                    if sim > similarity_threshold:
                        person_folder = os.path.join(output_dir, folder_name)
                        match_found = True
                        break

                if not match_found:
                    person_id_counter += 1
                    folder_name = f"person_{person_id_counter}"
                    person_folder = os.path.join(output_dir, folder_name)
                    os.makedirs(person_folder, exist_ok=True)
                    known_embeddings.append((emb, folder_name))

                crop = frame[y1:y2, x1:x2]
                face_filename = f"face_{frame_idx}_{i}.jpg"
                cv2.imwrite(os.path.join(person_folder, face_filename), crop)
                saved_count += 1

        frame_idx += 1
        pbar.update(1)

cap.release()
print(f"✅ Finished. Saved {saved_count} face crops into {person_id_counter} unique person folders.")

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

Processing video: 100%|██████████| 2000/2000 [01:24<00:00, 23.59it/s]


✅ Finished. Saved 532 face crops into 73 unique person folders.


Stricter params to avoid side profiles

In [9]:
import cv2
import os
import numpy as np
from insightface.app import FaceAnalysis
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# === Configuration ===
video_path = "/content/gdrive/MyDrive/Project Anthro/Face Recognition/input.mp4"
output_dir = "/content/gdrive/MyDrive/Project Anthro/Face Recognition/faces"
frame_interval = 10
similarity_threshold = 0.15
person_id_counter = 0
known_embeddings = []  # [(embedding, folder_name)]

# Setup face detector (GPU)
app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
app.prepare(ctx_id=0)

# Create output folder
os.makedirs(output_dir, exist_ok=True)

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("❌ ERROR: Could not open video.")

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"✅ Total frames in video: {total_frames}")

frame_idx = 0
saved_count = 0

# Progress bar
with tqdm(total=total_frames, desc="Processing video") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            height, width = frame.shape[:2]
            faces = app.get(frame)

            for i, face in enumerate(faces):
                x1, y1, x2, y2 = map(int, face.bbox)
                if x1 < 0 or y1 < 0 or x2 > width or y2 > height:
                    continue

                # === Orientation check: skip back or side-profile faces ===
                kps = face.kps
                if kps is None or len(kps) < 5:
                    continue  # Landmark extraction failed → likely turned away or profile

                left_eye, right_eye = kps[0], kps[1]
                eye_dist = np.linalg.norm(left_eye - right_eye)
                if eye_dist < 10:
                    continue  # Eyes too close or overlapping → likely only one eye visible → side/back view
                # === End orientation check ===

                # Get embedding
                emb = face.embedding.reshape(1, -1)
                match_found = False

                for known_emb, folder_name in known_embeddings:
                    sim = cosine_similarity(emb, known_emb.reshape(1, -1))[0][0]
                    if sim > similarity_threshold:
                        person_folder = os.path.join(output_dir, folder_name)
                        match_found = True
                        break

                if not match_found:
                    person_id_counter += 1
                    folder_name = f"person_{person_id_counter}"
                    person_folder = os.path.join(output_dir, folder_name)
                    os.makedirs(person_folder, exist_ok=True)
                    known_embeddings.append((emb, folder_name))

                crop = frame[y1:y2, x1:x2]
                face_filename = f"face_{frame_idx}_{i}.jpg"
                cv2.imwrite(os.path.join(person_folder, face_filename), crop)
                saved_count += 1

        frame_idx += 1
        pbar.update(1)

cap.release()
print(f"✅ Finished. Saved {saved_count} face crops into {person_id_counter} unique person folders.")

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

Processing video: 100%|██████████| 2000/2000 [01:22<00:00, 24.38it/s]

✅ Finished. Saved 532 face crops into 14 unique person folders.


Step-by-step identificaion, simply extracting all faces

In [34]:
import os
import json

prj_dir = "/content/gdrive/MyDrive/Project Anthro/Face Recognition/"
test_file = os.path.join(prj_dir, "write_test.json")

try:
    # Try creating and writing a small JSON file
    with open(test_file, "w") as f:
        json.dump({"test": "success"}, f)
    print("✅ JSON write test successful.")


except Exception as e:
    print("❌ Failed to write to JSON file.")
    print("Error:", e)

✅ JSON write test successful.


In [36]:
import cv2
import os
import numpy as np
from insightface.app import FaceAnalysis
from tqdm import tqdm
import json

results = []

prj_dir = "/content/gdrive/MyDrive/Project Anthro/Face Recognition/"
video_path = os.path.join(prj_dir, "input.mp4")
output_dir = os.path.join(prj_dir, "all-faces-2")
frame_interval = 10

# Setup face detector (GPU)
app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
app.prepare(ctx_id=0)

# Create output folder
os.makedirs(output_dir, exist_ok=True)

# Open video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise RuntimeError("ERROR: Could not open video.")

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_idx = 0
saved_count = 0

with tqdm(total=total_frames, desc="Processing video") as pbar:

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:

            height, width = frame.shape[:2]
            faces = app.get(frame)

            for i, face in enumerate(faces):
                x1, y1, x2, y2 = map(int, face.bbox)

                # Skip if box is out of frame
                if x1 < 0 or y1 < 0 or x2 > width or y2 > height:
                    continue

                # Orientation check: landmarks required
                kps = face.kps
                if kps is None or len(kps) < 2:
                    continue

                # Crop and save face
                crop = frame[y1:y2, x1:x2]
                face_filename = f"face_{frame_idx}_{i}.jpg"
                face_path = os.path.join(output_dir, face_filename)
                cv2.imwrite(face_path, crop)
                saved_count += 1

                # Compute normalized eye distance
                left_eye = kps[0]
                right_eye = kps[1]
                euclidean_distance = np.linalg.norm(left_eye - right_eye)
                bbox_width = x2 - x1
                normalized_eye_distance = euclidean_distance / bbox_width

                normalized_lip_distance = None
                if len(kps) >= 5:
                    left_lip = kps[3]
                    right_lip = kps[4]
                    lip_distance = np.linalg.norm(left_lip - right_lip)
                    normalized_lip_distance = lip_distance / bbox_width

                # Calculate nose placement relative to eyes
                nose_position = None
                if len(kps) >= 3:
                    nose = kps[2]
                    if nose[0] < min(left_eye[0], right_eye[0]):
                        nose_position = -1
                    elif nose[0] > max(left_eye[0], right_eye[0]):
                        nose_position = 1
                    else:
                        nose_position = 0

                result = {
                    "filename": face_filename,
                    "normalized_eye_distance": float(normalized_eye_distance),
                    "normalized_lip_distance": float(normalized_lip_distance) if normalized_lip_distance is not None else None,
                    "det_score": float(face.det_score),
                    "orientation_magnitude": None,
                    "mask": None,
                    "nose_placement": nose_position
                }

                if hasattr(face, 'pose') and face.pose is not None:
                    yaw, pitch, roll = face.pose
                    orientation_magnitude = np.linalg.norm([yaw, pitch, roll])
                    result["orientation_magnitude"] = float(orientation_magnitude)

                if hasattr(face, 'mask'):
                    result["mask"] = face.mask

                results.append(result)


        frame_idx += 1
        pbar.update(1)

cap.release()
print(f"Finished. Saved {saved_count} face crops to {output_dir}\n")

# Save results to JSON file in prj_dir
json_output_path = os.path.join(prj_dir, "face_metrics.json")

# Save or create JSON file
if os.path.exists(json_output_path):
    with open(json_output_path, "r") as f:
        existing_data = json.load(f)
    existing_data.extend(results)
    with open(json_output_path, "w") as f:
        json.dump(existing_data, f, indent=4)
else:
    with open(json_output_path, "w") as f:
        json.dump(results, f, indent=4)

print(f"Face metrics saved to {json_output_path}")

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

Processing video: 100%|██████████| 2000/2000 [01:23<00:00, 24.06it/s]

Finished. Saved 573 face crops to /content/gdrive/MyDrive/Project Anthro/Face Recognition/all-faces-2

Face metrics saved to /content/gdrive/MyDrive/Project Anthro/Face Recognition/face_metrics.json


In [22]:
print('hi')

hi
